In [1]:
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
import streamlit as st

In [2]:
client = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d"
)

In [3]:
user_query = "최저시급 대해 정보를 출력해줘"

In [4]:
import os
import openai
import tiktoken

os.environ["OPENAI_API_KEY"] = 'API_KEY'

tokenizer = tiktoken.get_encoding("cl100k_base")

# 토크나이저 기반으로 토큰 개수를 새줌
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)


from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pages = loader.load_and_split()

# chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

# text embedding
# embedding model
from langchain.embeddings import HuggingFaceEmbeddings

model_name ="jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# texts들에 대해 hf 임베딩 모델로 임베딩 하는 과정 -> 이를 docsearch에 저장
docsearch = Chroma.from_documents(texts, hf)

c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
docsearch

In [6]:
llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1)

c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
qa = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type= "stuff",
                                 retriever = docsearch.as_retriever(
                                     search_type="mmr",
                                     search_kwargs={'k':3, 'fetch_k': 10}),
                                     return_source_documents=True)

In [8]:
# def create_prompt_template(question: str, context: str) -> str:
#     prompt_template = f"""
#     You are an AI assistant. Answer the following question in detail and in Korean.

#     Question: {question}

#     Context: {context}

#     Answer:
#     """
#     return prompt_template

In [9]:
# def chat_with_bot(question):
#     # 관련 문서 검색
#     response = qa({"query": question})
#     context = " ".join([doc.page_content for doc in response['source_documents']])
    
#     # 프롬프트 템플릿 생성
#     prompt = create_prompt_template(question, context)
    
#     # 메시지 형식으로 전달
#     messages = [
#         {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer in korean."},
#         {"role": "user", "content": prompt}
#     ]
    
#     # 응답 생성
#     chat_response = client.chat.completions.create(messages)
    
#     # 출력
#     print("Answer:", chat_response['choices'][0]['message']['content'])
#     print("\nSources:")
#     for source in response['source_documents']:
#         print(source.metadata['source'])
#         print(source.page_content)
#         print("\n---\n")

# # 예시 질문
# question = "근로조건을 낮출 수 있나요?"
# chat_with_bot(question)

In [10]:
# def chat_with_bot(question):
#     response = qa({"query": question})
#     answer = response['result']
#     sources = response['source_documents']

#     문서 내용을 합쳐서 하나의 컨텍스트로 만들기
    
#     # 출력
#     print("Answer:", answer)
#     print("\nSources:")
#     for source in sources:
#         print(source.metadata['source'])
#         print(source.page_content)
#         print("\n---\n")

# # 예시 질문
# question = "근로조건을 낮출 수 있나요?"
# chat_with_bot(question)

In [11]:
def create_prompt_template(question):
    template = (
        "당신은 한국 노동법에 대한 전문 지식을 가진 AI 어시스턴트입니다. "
        "다음 질문에 대해 주어진 문맥을 사용하여 상세하고 정확한 답변을 제공해주세요. "
        "관련 문맥 부분을 참조하여 답변을 작성하세요.\n\n"
        "질문: {question}\n\n"
        "답변:"
    )
    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template=template
    )
    return prompt_template.format(question=question)

# chat_with_bot 함수 수정
def chat_with_bot(question):
    # 질문에 대한 응답 및 관련 문서 검색
    response = qa({"query": question})
    answer = response['result']
    sources = response['source_documents']
    
    # 프롬프트 생성
    prompt = create_prompt_template(question)
    
    # client.chat.completions.create를 사용하여 응답 생성
    response = client.chat.completions.create(
        model="xionic-ko-llama-3-70b",
        messages=[
            {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer in korean."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # 출력
    print("\n=== Answer ===")
    print(response)
    
    print("\n=== Sources ===")
    for i, source in enumerate(sources, 1):
        print(f"\n--- Source {i} ---")
        print(f"Source File: {source.metadata['source']}")
        print(f"Page Number: {source.metadata.get('page', 'N/A')}")
        print(source.page_content)

# 예시 질문
question = "휴가를 내고 여행가고 싶은데 얼마나 휴가 내는 것이 가능할까?"
chat_with_bot(question)

c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [20]:
# result = qa(user_query)
# result

{'query': '최저시급 대해 정보를 출력해줘',
 'result': 'I apologize, but there is no information about the minimum wage (최저시급) in the provided context. The context appears to be related to labor laws and regulations in Korea, but it does not mention the minimum wage. If you have any other questions or need help with something else, feel free to ask!',
 'source_documents': [Document(metadata={'page': 7, 'source': 'C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf'}, page_content='임금을 보장하여야 한다.\n \n제48조(임금대장 및 임금명세서) ① 사용자는 각 사업장별로 임금대장을 작성하고 임금과 가족수당 계산의 기초가 되\n는 사항, 임금액, 그 밖에 대통령령으로 정하는 사항을 임금을 지급할 때마다 적어야 한다. <개정 2021. 5. 18.>\n② 사용자는 임금을 지급하는 때에는 근로자에게 임금의 구성항목ㆍ계산방법, 제43조제1항 단서에 따라 임금의 일\n부를 공제한 경우의 내역 등 대통령령으로 정하는 사항을 적은 임금명세서를 서면(「전자문서 및 전자거래 기본법」\n제2조제1호에 따른 전자문서를 포함한다)으로 교부하여야 한다.<신설 2021. 5. 18.>\n[제목개정 2021. 5. 18.]\n \n제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.\n \n       제4장 근로시간과 휴식\n \n제50조(근로시간) ① 1주 간의 근로시간은 휴게시간을 제외하고 40시간을 초과할 수 없다.\n② 1일의 근로시간은 휴게시간을 제외하고 8시간을 초